In [19]:
import os

import cv2
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

from pathlib import Path
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.applications.vgg16 import VGG16

In [2]:
model = keras.models.load_model('trained_model.keras')

In [3]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2_front │ (None, 400, 300,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3_side  │ (None, 400, 300,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_front  │ (None, 400, 300,  │      1,792 │ input_layer_2_fr… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_side   │ (None, 400, 300,  │      1,792 │ input_layer_3_si… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_front  │ (None, 400, 300,  │     36,928 │ block1_conv1_fro… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_side   │ (None, 400, 300,  │     36,928 │ block1_conv1_sid… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_pool_front   │ (None, 200, 150,  │          0 │ block1_conv2_fro… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_pool_side    │ (None, 200, 150,  │          0 │ block1_conv2_sid… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_conv1_front  │ (None, 200, 150,  │     73,856 │ block1_pool_fron… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_conv1_side   │ (None, 200, 150,  │     73,856 │ block1_pool_side… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_conv2_front  │ (None, 200, 150,  │    147,584 │ block2_conv1_fro… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_conv2_side   │ (None, 200, 150,  │    147,584 │ block2_conv1_sid… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_pool_front   │ (None, 100, 75,   │          0 │ block2_conv2_fro… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_pool_side    │ (None, 100, 75,   │          0 │ block2_conv2_sid… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv1_front  │ (None, 100, 75,   │    295,168 │ block2_pool_fron… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv1_side   │ (None, 100, 75,   │    295,168 │ block2_pool_side… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv2_front  │ (None, 100, 75,   │    590,080 │ block3_conv1_fro

 Total params: 40,052,741 (152.79 MB)

 Trainable params: 3,541,121 (13.51 MB)

 Non-trainable params: 29,429,376 (112.26 MB)

 Optimizer params: 7,082,244 (27.02 MB)

In [45]:
def load_image_pairs(directory, label):
    pairs = []
    subdirs = [d for d in directory.iterdir() if d.is_dir()]
    for subdir in subdirs:
        images = sorted(list(subdir.glob('*.jpeg')))
        if len(images) == 2:
            front_img = load_grayscaled_image(images[0])
            side_img = load_grayscaled_image(images[1])
            pairs.append((front_img, side_img, label, subdir.name))
    return pairs

def load_grayscaled_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (target_size[1], target_size[0]))
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    img = img / 255.
    img = img.astype('float32')
    return np.array(img)

In [5]:
target_size = (400,300)

In [46]:
data_path = Path('./test_for_heatmap')
test_data = load_image_pairs(data_path,1)

In [47]:
X_front = np.array([pair[0] for pair in test_data])
X_side = np.array([pair[1] for pair in test_data])
subdir_names = [pair[3] for pair in test_data]

In [48]:
subdir_names[0]

'48'

In [8]:
X_front.shape

(9, 400, 300, 3)

In [17]:
def record_grad_tape(partial_model, classifier_model, X):
    with tf.GradientTape() as tape:
        # 入力をTensorFlowテンソルに変換
        partial_last_conv_layer_output = partial_model(tf.expand_dims(X, axis=0))
        tape.watch(partial_last_conv_layer_output)

        preds = classifier_model(partial_last_conv_layer_output)
        top_class_channel = preds[:, 0]  # クラス0に対応するチャンネル

        # 勾配計算
        grads = tape.gradient(top_class_channel, partial_last_conv_layer_output)

    # チャネルごとの平均勾配
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2)).numpy()

    # ヒートマップ生成
    partial_last_conv_layer_output = partial_last_conv_layer_output.numpy()[0]
    for i in range(pooled_grads.shape[-1]):
        partial_last_conv_layer_output[:, :, i] *= pooled_grads[i]

    heatmap = np.mean(partial_last_conv_layer_output, axis=-1)
    heatmap = np.maximum(heatmap, 0)  # 負の値を0に変換
    heatmap /= np.max(heatmap)  # 正規化

    return heatmap

In [10]:
vgg_front = VGG16(weights='imagenet', include_top=False, input_shape=(target_size[0],target_size[1], 3), name='front_vgg16')
for layer in vgg_front.layers:
    layer.trainable = False
    layer.name = layer.name + '_front'

In [11]:
vgg_side = VGG16(weights='imagenet', include_top=False, input_shape=(target_size[0], target_size[1], 3), name='side_vgg16')
for layer in vgg_side.layers:
    layer.trainable = False
    layer.name = layer.name + '_side'

In [20]:
# 最後の畳み込み層を取得
front_last_conv_name = 'block5_conv3_front'
front_last_conv_layer = vgg_front.get_layer(front_last_conv_name)

# 部分モデル
front_last_conv_layer_model = tf.keras.Model(
    inputs=vgg_front.input,
    outputs=front_last_conv_layer.output,
    training=False
)

# クラス分類モデル
front_classifier_input = layers.Input(
    shape=front_last_conv_layer.output.shape[1:]
)
front = front_classifier_input
front_classifier_layer_names = ['block5_pool_front']
for layer_name in front_classifier_layer_names:
    front = vgg_front.get_layer(layer_name)(front)

front_classifier_model = models.Model(front_classifier_input, front)

In [14]:
print(front_last_conv_layer_model.dtype)
print(vgg_front.input.shape)

float32
(None, 400, 300, 3)


In [21]:
front_last_conv_layer_model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_front (InputLayer)  │ (None, 400, 300, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1_front (Conv2D)     │ (None, 400, 300, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2_front (Conv2D)     │ (None, 400, 300, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool_front               │ (None, 200, 150, 64)   │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1_front (Conv2D)     │ (None, 200, 150, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2_front (Conv2D)     │ (None, 200, 150, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool_front               │ (None, 100, 75, 128)   │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1_front (Conv2D)     │ (None, 100, 75, 256)   │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2_front (Conv2D)     │ (None, 100, 75, 256)   │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3_front (Conv2D)     │ (None, 100, 75, 256)   │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool_front               │ (None, 50, 37, 256)    │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1_front (Conv2D)     │ (None, 50, 37, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2_front (Conv2D)     │ (None, 50, 37, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3_front (Conv2D)     │ (None, 50, 37, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool_front               │ (None, 25, 18, 512)    │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1_front (Conv2D)     │ (None, 25, 18, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2_front (Conv2D)     │ (None, 25, 18, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3_front (Conv2D)     │ (None, 25, 18, 512)    │     2,359,808 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

In [37]:
# 最後の畳み込み層を取得
side_last_conv_name = 'block5_conv3_side'
side_last_conv_layer = vgg_side.get_layer(side_last_conv_name)

# 部分モデル
side_last_conv_layer_model = tf.keras.Model(
    inputs=vgg_side.input,
    outputs=side_last_conv_layer.output,
    training=False
)

# クラス分類モデル
side_classifier_input = layers.Input(
    shape=side_last_conv_layer.output.shape[1:]
)
side = side_classifier_input
side_classifier_layer_names = ['block5_pool_side']
for layer_name in side_classifier_layer_names:
    side = vgg_side.get_layer(layer_name)(side)

side_classifier_model = models.Model(side_classifier_input, side)

In [62]:
def save_heatmap(heatmap, X, i, save_path):
    img = X * 255
    heatmap = np.uint8(255 * heatmap)
    
    jet = cm.get_cmap('jet')
    jet_colors = jet(np.arange(256))[:,:3]
    jet_heatmap = jet_colors[heatmap]
    
    jet_heatmap = keras.utils.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.utils.img_to_array(jet_heatmap)
    
    superimposed_img = jet_heatmap * 0.3 + img
    superimposed_img = keras.utils.array_to_img(superimposed_img)

    root_dir = Path(f'./heatmap/{subdir_names[i]}/')
    root_dir.mkdir(exist_ok=True, parents=True)
    save_path = Path(save_path)
    
    superimposed_img.save(root_dir / save_path)

In [63]:
for i in range(len(X_front)):
    front_heatmap = record_grad_tape(front_last_conv_layer_model, front_classifier_model, X_front[i])
    save_heatmap(front_heatmap, X_front[i], i, f'front_cmap_{subdir_names[i]}.jpeg')
    
    side_heatmap = record_grad_tape(side_last_conv_layer_model, side_classifier_model, X_side[i])
    save_heatmap(side_heatmap, X_side[0], i, f'side_cmap_{subdir_names[i]}.jpeg')

/var/folders/6k/f84lxj_n4zv0qkpmgkdwrk000000gn/T/ipykernel_19933/3908318452.py:5: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  jet = cm.get_cmap('jet')
